In [3]:
import com.ktsdk.GeminiApi
import com.ktsdk.MimeType
import com.ktsdk.Models
import com.ktsdk.models.InlineData
import com.ktsdk.models.Part
import com.ktsdk.tools.FunctionSchemaGenerator
import com.ktsdk.tools.GeminiFunction
import com.ktsdk.tools.GeminiParameter

val model = Models.GEMINI_2_5_FLASH.modelName
val liveModel = Models.GEMINI_2_5_FLASH_LIVE_PREVIEW.modelName

val apiKey = System.getenv("GEMINI_API_KEY")

val geminiApi = GeminiApi(apiKey = apiKey, model = model)

val systemInstruction = "sample system instruction"
val filesPath = listOf("/path/file1.txt", "/path/file2.txt", "/path/file3.txt")
val prompt = "this is a prompt"

//these are suspend function so use of them in the coroutine scope or another suspend function
geminiApi.tokenCount(
    systemInstruction = systemInstruction,
    filesPath = filesPath,
    prompt = prompt
)

val url = "sample content url"
//streamed
geminiApi.streamGenerateContent(
    url = url,
    systemInstruction = systemInstruction,
    prompt = prompt
) { response ->
    println(response)
}

val message = listOf(
    Part(
        text = prompt,
        inlineData = InlineData(mimeType = MimeType.JPEG.mime, data = "image/path")
    )
)
//Non-Streamed
geminiApi.generateContentWithTool(
    systemInstruction = systemInstruction,
    prompts=message,
    googleSearchTool=true
){response ->
    println(response)
}

//create function for function calling

@GeminiFunction(description = "description of the function")
fun sayHello(
    @GeminiParameter(description = "description of the parameter")
    name:String,
): String {
    return "sample return guider for more help to best using, say 'hello $name'"
}

val helloFunction = FunctionSchemaGenerator.generateSchema(::sayHello)

//use function calling
geminiApi.streamGenerateContent(
url = url,
    systemInstruction = systemInstruction,
    prompt = prompt,
    functionCall = listOf(helloFunction),
    googleSearchTool = true
){response ->
    println(response)
}

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[3], line 59, column 23: Parameter name expected